In [1]:
from PdfCleaner import PdfCleaner
from os import listdir
from os.path import isfile, join
from PerformanceTesting import PerformanceTester
import pdfplumber
import json
import pandas as pd
import numpy as np
import requests
from pymongo import MongoClient


class Transcripts:
    def __init__(self, ticker):
        self.ticker = ticker
        xinyu = "/Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project"
        marco = "C:/Users/mtort/Repositories/DS3500-Final-Project"
        emily = "/Users/emilywang/Desktop/DS3500-Final-Project-main-2"
        kelly = "/Users/kelly/Desktop/ds3500/DS3500-Final-Project"
        qi = ""
        self.path = xinyu + "/transcripts/"+ticker+"_transcripts/"

    def read_files(self):
        return [f for f in listdir(self.path) if isfile(join(self.path, f))]

    def create_dct(self):
        lst_files = self.read_files()
        lst_cleaned = []
        for file in lst_files:
            file_path = self.path + file
            if file_path[-3:] == 'pdf':
                txt = PdfCleaner(file_path)
                date = file[:8]
                # txt_cleaned = txt.clean_stopwords_punctuation()
                txt_cleaned = txt.clean_nums()
                PerfTest = PerformanceTester()
                PerfTest.setTimeframe('day', 1)
                PerfTest.loadArticles([[self.ticker, date, txt_cleaned]])
                try:
                    classification_xy = PerfTest.aquireTargetValues()
                except KeyError:
                    print(f'''Warning: attempted to access market during weekend or after hours
                          Earnings Transcript {file_path} Not Added
                          ''')
                    continue
                dct_cleaned = {'price_change': classification_xy[1][0], 'name': self.ticker, 'date': date, 'transcript': txt_cleaned}
                lst_cleaned.append(dct_cleaned)
        return lst_cleaned

class Database:
    def __init__(self):
        client = MongoClient()
        client.drop_database('transcripts')
        self.db = client.transcripts

    def store_data(self, tickers_lst):
        for t in tickers_lst:
            store = Transcripts(t)
            transcript = store.create_dct()
            self.db.transcript.insert_many(transcript)
            print(t + " transcripts stored successfully")
        return self.db
    

In [15]:
''' To prevent "IOPub data rate exceeded error":
enter into terminal: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 '''

tickers = ['AAPL', 'FB', 'MSFT', 'GOOGL', 'NFLX', 'TSLA', 'ADBE', 'CMCSA', 'COST', 'AMZN']
    # works: APPL, MSFT
    # doesn't work: FB, GOOGL, NFLX, TSLA, ADBE, CMCSA, COST, AMZN

    # 'APPL', 'MSFT', 'FB', 'GOOGL', 'NFLX', 'TSLA', 'ADBE', 'CMCSA', 'COST', 'AMZN'
    # client = MongoClient()
    # client.drop_database('transcripts')
    # db = client.transcripts


    # for t in tickers:
    #     store = Transcripts(t)
    #     transcript = store.create_dct()
    #     # print(transcript)
    #     db.transcript.insert_many(transcript)
    #     print(t + " transcripts stored successfully")

    # db.transcript.find() # client.transcripts.transcript.find()

database = Database()
db = database.store_data(tickers)
# all_transcripts = db.transcript.find()
# for transcript in all_transcripts:
#     print(transcript)

                          Earnings Transcript /Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526178.pdf Not Added
                          
                          Earnings Transcript /Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526172.pdf Not Added
                          
AAPL transcripts stored successfully
FB transcripts stored successfully
MSFT transcripts stored successfully
GOOGL transcripts stored successfully
                          Earnings Transcript /Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project/transcripts/NFLX_transcripts/20190118_Netflix_Inc-_Earnings_Call_2019-1-17_DN000000002573718691.pdf Not Added
                          
NFLX transcripts stored successfully
TSLA transcripts stored successfully
ADBE transcripts stored successfully
CMCSA transcripts sto

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


''' Query to test on all APPLE transcripts '''
# apple = db.transcript.find({"name":"APPL"})


''' Query to test on all transcripts '''
all_companies = db.transcript.find()

data = {}
text = []

for transcript in all_companies:
    data[transcript["name"]+" "+transcript["date"]] = transcript["transcript"]
    text.append(transcript["transcript"])
#     print(transcript)
#     df = pd.DataFrame.from_dict(data, orient ='index')




# for name, info in data.items():
''' Use countvectorizer to convert pdf text to a matrix of token counts '''
# vect = CountVectorizer()
# vect.fit(text)
#     print("vocabulary: \n", vect.vocabulary_)


''' Use bag of words to count how many times each word appears in pdf '''
# bag_of_words = vect.transform(text)
#     print(bag_of_words)

    # number of times each unique word appears in this text
# bag_of_words_arr = bag_of_words.toarray()
# #     print(bag_of_words_arr)
# feature_names = vect.get_feature_names()
#     print(feature_names)

''' Display occurrences in dataframe '''
# bow_df = pd.DataFrame(bag_of_words_arr, columns = feature_names)
# bow_df["Transcript (Company)"] = "APPL"
# bow_df.set_index("Transcript (Company & Date)", inplace=True)

''' Use Tfidf vectorizer to match words to TFIDF values '''

vect = TfidfVectorizer(min_df=3, ngram_range = (1, 1)).fit(text)
bag_of_words = vect.transform(text)
feature_names = vect.get_feature_names()
    
tfidf_df = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
# tfidf_df[name] = text
# tfidf_df.set_index(name, inplace=True)

# tfidf_df.head()
    
    
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(bag_of_words_arr).toarray()

    
# tfidf_df = pd.DataFrame(X, columns = feature_names)
# tfidf_df["Transcript (Company)"] = "APPL"
# tfidf_df.set_index("Transcript (Company & Date)", inplace=True)

# ''' Show number of occurences of each word in one dataframe and tfidf (weighted) values of each word in another
#     dataframe for each pdf '''
# display(bow_df)
display(tfidf_df)

,aaa,aapl,aaron,ab,abate,abb,abc,abcxyzinvestor,abhay,abide,...,ﬂuctuate,ﬂuctuates,ﬂuctuation,ﬂuctuations,ﬂuid,ﬂuidity,ﬂush,ﬂux,ﬂy,ﬂywheel
0,0.0,0.164058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.014508,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.150070,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.006768,0.00992,0.0,0.0,0.0,0.0,0.0
2,0.0,0.089964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.088728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.163487,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.011812,0.00000,0.0,0.0,0.0,0.0,0.0
271,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.013892,0.0,0.0,0.010641,0.00000,0.0,0.0,0.0,0.0,0.0
272,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0
273,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.016804,0.0,0.0,0.012871,0.00000,0.0,0.0,0.0,0.0,0.0


In [79]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

''' Using PCA to reduce total number of features before feeding into ML model '''
# instantiate the PCA object and request reduced number of components (reduces number of columns/features)
pca = PCA(n_components=150, random_state=3000)


# standardize the features so they are all on the same scale
features_standardized = StandardScaler().fit_transform(bag_of_words.toarray())

# transform the standardized features using the PCA algorithm 
reduced_data = pca.fit_transform(features_standardized)

# show transformed results in dataframe
# components = ["Component1", "Component2", "Component3", "Component4", "Component5", "Component6",
#              "Component7", "Component8", "Component9", "Component10"]

pca_df = pd.DataFrame(reduced_data)#, columns = components)

''' Obtain target values (whether stock price increased, decreased, or stayed the same) 
from database '''
price_changes = []
all_transcripts = db.transcript.find()
for transcript in all_transcripts:
    price_changes.append(transcript['price_change'])

pca_df['target'] = price_changes 

pca_df



,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,target
0,-2.307979,2.528321,0.904425,12.432409,51.007731,-23.395203,-15.809925,-4.318997,2.183633,-2.517567,...,0.218535,1.671613,0.511702,0.850236,-0.633378,0.050788,-0.858868,1.183464,0.247281,0
1,-1.614576,0.922218,1.169598,5.935298,39.845241,-16.709913,-10.535552,-3.382118,4.307920,-1.998716,...,13.735506,-4.008181,-9.967395,12.687086,-5.967942,-7.868016,-7.701840,-9.523958,-0.680621,0
2,0.989392,-6.101740,4.182001,11.004364,31.341852,-12.311494,-14.064921,1.813661,-8.861864,-5.199981,...,0.751213,0.246224,0.203585,-2.149053,1.007944,1.196486,0.062013,-0.585796,-0.693569,1
3,4.525036,-6.524596,5.407076,8.773479,25.481568,-10.590066,-12.147817,2.731886,-12.901776,-5.425576,...,-0.152851,1.209940,1.492534,-1.278281,0.850205,-1.443709,1.256342,0.089530,-2.213685,1
4,-2.829213,3.603864,1.417274,5.726625,43.837194,-20.823234,-10.200290,-6.212460,4.141825,-3.100824,...,-2.370975,3.454321,-0.332447,2.867899,2.284075,-0.640776,0.677898,1.054457,1.872206,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,-1.820701,24.543798,-16.264420,-8.784679,3.772257,-9.097212,36.405299,6.211238,-8.838499,-5.080124,...,2.065213,0.255603,-0.010978,2.589003,0.642645,2.643024,-6.795618,2.407337,2.845705,0
271,-0.321400,18.336721,-12.670496,-6.245048,4.305022,-6.876958,29.328610,6.466410,-15.745458,-6.357559,...,-1.339970,-0.734529,-0.498698,0.962309,2.682003,-2.348483,1.277172,0.490272,-0.470459,1
272,4.014123,7.818487,-6.579952,-0.612990,2.577400,-4.100341,18.741163,9.303537,-25.995166,-8.055968,...,-2.001872,-0.811732,-1.204700,0.463712,3.134725,-1.705194,0.466016,-0.885878,-0.515039,1
273,-3.337460,25.779128,-14.402769,-6.126839,4.782124,-10.000960,38.981290,6.129891,-10.679813,-6.785669,...,2.081628,5.794964,1.106284,6.689669,2.785931,3.555687,5.165071,8.078172,0.165094,1


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

features = pca_df.drop("target", axis = 1)
target = pca_df["target"]

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

#initialize classifier
clf = LinearSVC(random_state=3000, max_iter=1000000)

#create the model by fitting the training data
clf.fit(X=X_train, y=y_train)

#prediction accuracy
accuracy = clf.score(X_test, y_test)

print("Prediction accuracy on the test data:", f"{accuracy:.2%}")

Prediction accuracy on the test data: 79.71%


In [81]:
from sklearn.linear_model import LogisticRegression

#split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=3000)

#initialize classifier
log = LogisticRegression(random_state=3000)

#create the model by fitting the training data
log.fit(X=X_train, y=y_train)

#prediction accuracy
accuracy = log.score(X_test, y_test)

print("Prediction accuracy on the test data:", f"{accuracy:.2%}")

Prediction accuracy on the test data: 78.26%


/Users/xinyuwu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
features

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-3.480583,-1.567485,6.324731,-3.885820,62.475400,-19.032408,-17.879378,-5.289889,-2.675863,-2.657930,...,0.198032,0.252274,0.978913,-0.066256,0.726172,0.182786,0.044244,-0.920215,0.244012,0.037977
1,-2.818895,-1.303089,1.251143,-0.219564,42.070870,-12.016005,-8.809017,-3.229944,0.344175,-6.180802,...,-15.214914,15.227972,6.750283,-0.790302,0.836719,4.954599,-1.900787,6.164296,-6.878360,5.922553
2,0.203164,5.745502,5.289153,-0.877182,39.283276,-8.551421,-16.313376,-0.967650,-7.390542,4.542740,...,-0.347261,0.943549,-0.290801,-0.895316,0.636850,-1.130229,0.727421,0.125277,-0.208713,-0.048605
3,3.890170,5.676963,4.608314,1.446412,32.094591,-7.279872,-14.598872,-0.581982,-9.467259,9.655062,...,0.433578,0.674606,-0.309290,0.185601,-0.200346,-1.739470,0.036334,-0.457965,-0.195351,0.686561
4,-4.177812,-3.353393,1.820723,0.216489,50.415855,-17.037638,-9.689804,-6.912139,-1.573690,-5.415118,...,-0.507211,-1.550773,0.694427,1.955418,-1.413462,0.076644,-0.136668,1.100726,0.303579,-0.563170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,-3.862068,-22.318476,-6.646760,-7.913091,1.594886,-11.911691,38.015915,6.219002,-12.469402,7.438955,...,1.636520,6.720818,8.458539,7.627001,10.793185,2.548774,-0.988733,0.807389,-1.620166,11.472889
271,-1.998209,-17.366611,-5.372135,-6.529522,2.765056,-9.009398,31.605272,5.491215,-16.630762,14.534642,...,-0.661223,-0.360966,2.469537,1.466760,-0.008199,0.390574,-3.427715,1.158981,1.794620,-0.933308
272,2.706552,-7.934565,-1.576781,-4.471130,2.366276,-4.731855,19.271655,5.993271,-20.013113,22.628913,...,-0.090617,-0.410573,1.546547,1.716508,0.056506,0.402579,-3.782575,1.916585,1.854015,-0.936546
273,-5.305766,-22.671868,-3.460939,-7.662288,2.925618,-11.980639,40.446724,5.674845,-14.707235,8.941162,...,-8.357154,-17.182060,6.659348,4.975648,-5.437035,-2.427682,13.894293,2.884586,-0.208268,2.820466
